#Setup

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
!python -m spacy download zh_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 39.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_trf')


In [ ]:
import spacy

#**Make df from sample text**

In [ ]:
nlp =spacy.load('zh_core_web_trf')

In [ ]:
%%writefile chinese_text_demo.txt
'''
好,這邊有一個小孩子玩
這個叫什麼? 鮷魚喔
鮷魚的那個遊戲
那這邊就是一個老闆在擺麵,擺那個飲食攤
他利用這個說他們跟幾個小孩在那邊玩Sparat
如果我猜的,也許小孩子贏老闆的話
獎品就是這些吃的東西
大概是這樣
然後這個媽媽手拿的冰淇淋她也忘記了
給小孩吃,然後小孩的冰淇淋就滴到
'''

Writing chinese_text_demo.txt


# General Methods

In [ ]:
def filterText(text):
  import nltk
  # this tokeniser takes care of contractions nicely
  from nltk.tokenize import WhitespaceTokenizer
  # Create a reference variable for Class WhitespaceTokenizer
  tk = WhitespaceTokenizer()
  trackWords = tk.tokenize(text)
  #removing unwanted characters, excluding contractions
  bad_chars = [',', ':', '!', '\"', '?']
  filteredTrackWords = [''.join(filter(lambda i: i not in bad_chars, word)) for word in trackWords]
  #remove empty words
  filteredTrackWords = [i for i in filteredTrackWords if i]
  return " ".join(filteredTrackWords)

In [ ]:
f = open("chinese_text_demo.txt","r")
doc = nlp(filterText(f.read()))

In [ ]:
def setupSpacyDF(doc):
  import pandas as pd
  cols = ("text",  "POS", "STOP", "DEP")
  rows = []
  for t in doc:
    if t.pos_ == 'PUNCT':
      #not considering punctuation
      continue
    row = [t.text, t.pos_, t.is_stop, t.dep_]
    rows.append(row)
  df = pd.DataFrame(rows, columns=cols)
  return df

In [ ]:
print(list(setupSpacyDF(doc)['text']))

['好', '這邊', '有', '一', '個', '小孩子', '玩', '這個', '叫', '什麼', '喔', '的', '那', '個', '遊戲', '那', '這邊', '就是', '一', '個', '老闆', '在', '擺麵', '擺', '那', '個飲', '食攤', '他', '利用', '這個', '說', '他們', '跟', '幾個', '小孩', '在', '那', '邊玩Sparat', '如果', '我', '猜', '的', '也', '許', '小孩子', '贏', '老闆', '的', '話', '獎品', '就是', '這些', '吃', '的', '東西', '大概', '是', '這樣', '然後', '這', '個媽', '媽手', '拿', '的', '冰淇淋', '她', '也', '忘記', '了', '給', '小孩', '吃然', '後', '小孩', '的', '冰淇淋', '就', '滴', '到']


#Easy feats

In [ ]:
def posTagCounts(df, linguisticFeatures):
  # 5.20.22: This line is problematic
  #tag_types = list(df['POS'].unique())
  #solution: scroll to bottom of https://stackoverflow.com/questions/58215855/how-to-get-full-list-of-pos-tag-and-dep-in-spacy
  tag_types = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "SCONJ", "SYM", "VERB", "X", "SPACE"]
  for category in tag_types:
    tag_description = "POS_TAG:" + category
    linguisticFeatures[tag_description] = (df['POS'] == category).sum()

In [ ]:
def openClosedRatio(df, linguisticFeatures):
  open_class = {'VERB', 'NOUN', 'ADJ', 'ADV'}
  tag_types = list(df['POS'].unique())
  open_words = 0
  closed_words = 0
  for category in tag_types:
    if category in open_class:
      open_words += (df['POS'] == category).sum()
    else:
      closed_words += (df['POS'] == category).sum()
  ratio = open_words/closed_words if closed_words else "NaN"
  linguisticFeatures["Open to Closed Words Ratio"] = ratio
# CHECK EDGE CASE


POS tags are consistent across languages

In [ ]:
def posNormalizedCounts(df, linguisticFeatures):
  #tag_types = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "SCONJ", "SYM", "VERB", "X", "SPACE"]
  filtered_tag_types = ["ADJ", "ADP", "ADV", "AUX","CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "SCONJ", "VERB"]
  #for category in tag_types:
  #DECIDE
  for category in filtered_tag_types:
    tag_description = "POS_TAG:" + category
    linguisticFeatures[tag_description] = (df['POS'] == category).sum()/df.shape[0]

#Dependency tags

In [ ]:
#source: https://spacy.io/models/en#en_core_web_trf-labels
englishDepTags = ['ROOT', 'acl', 'acomp','advcl','advmod','agent','amod','appos','attr','aux','auxpass','case','cc','ccomp','compound','conj','csubj','csubjpass','dative','dep','det','dobj','expl','intj','mark','meta','neg','nmod','npadvmod','nsubj','nsubjpass','nummod','oprd','parataxis','pcomp','pobj','poss','preconj','predet','prep','prt','punct','quantmod','relcl','xcomp']

In [ ]:
#source: https://spacy.io/models/zh#zh_core_web_trf-labels
chineseDepTags = ["ROOT", "acl", "advcl:loc", "advmod", "advmod:dvp", "advmod:loc","advmod:rcomp", "amod", "amod:ormod", "appos", "aux:asp", "aux:ba", "aux:modal", "aux:prtmod", "auxpass", "case", "cc", "ccomp", "compound:nn", "compound:vc", "conj", "cop", "dep", "det", "discourse", "dobj", "etc","mark","mark:clf", "name", "neg","nmod","nmod:assmod","nmod:poss","nmod:prep", "nmod:range", "nmod:tmod", "nmod:topic", "nsubj", "nsubj:xsubj", "nsubjpass","nummod", "parataxis:prnmod", "punct", "xcomp"]

In [ ]:
advmodSpecificChineseTags = ["advmod:dvp", "advmod:loc","advmod:rcomp"]
amodSpecificChineseTags = ["amod:ormod"]
auxSpecificChineseTags = ["aux:asp", "aux:ba", "aux:modal", "aux:prtmod","auxpass"]
compoundSpecificChineseTags = ["compound:nn", "compound:vc"]
markSpecificChineseTags = ["mark:clf"]
nmodSpecificChineseTags = ["nmod:assmod","nmod:poss","nmod:prep", "nmod:range", "nmod:tmod", "nmod:topic"]
nsubjSpecificChineseTags = ["nsubj:xsubj"]
nummodSpecificChineseTags = ["nummod"]

In [ ]:
chineseSimplifiedDepTags = []
for tag in chineseDepTags:
  if tag in advmodSpecificChineseTags:
    chineseSimplifiedDepTags.append("advmod")
  elif tag in amodSpecificChineseTags:
    chineseSimplifiedDepTags.append("amod")
  elif tag in auxSpecificChineseTags:
    chineseSimplifiedDepTags.append("aux")
  elif tag in compoundSpecificChineseTags:
    chineseSimplifiedDepTags.append("compound")
  elif tag in markSpecificChineseTags:
    chineseSimplifiedDepTags.append("mark")
  elif tag in nmodSpecificChineseTags:
    chineseSimplifiedDepTags.append("nmod")
  elif tag in nsubjSpecificChineseTags:
    chineseSimplifiedDepTags.append("nsubj")
  elif tag in nummodSpecificChineseTags:
    chineseSimplifiedDepTags.append("nummod")
  else:
    chineseSimplifiedDepTags.append(tag)

In [ ]:
chineseSimplifiedDepTags = set(chineseSimplifiedDepTags)
print(sorted(chineseSimplifiedDepTags))

['ROOT', 'acl', 'advcl:loc', 'advmod', 'amod', 'appos', 'aux', 'case', 'cc', 'ccomp', 'compound', 'conj', 'cop', 'dep', 'det', 'discourse', 'dobj', 'etc', 'mark', 'name', 'neg', 'nmod', 'nsubj', 'nsubjpass', 'nummod', 'parataxis:prnmod', 'punct', 'xcomp']


In [ ]:
print(sorted(englishDepTags))

['ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp']


In [ ]:
overlapChineseEnglishDepTags = chineseSimplifiedDepTags.intersection(set(englishDepTags))
print(overlapChineseEnglishDepTags)

{'case', 'amod', 'neg', 'advmod', 'dobj', 'acl', 'cc', 'det', 'conj', 'punct', 'nsubjpass', 'nummod', 'nsubj', 'mark', 'xcomp', 'ROOT', 'aux', 'dep', 'compound', 'appos', 'ccomp', 'nmod'}


In [ ]:
def depTagCounts(df, linguisticFeatures):
  overlapChineseEnglishDepTags = chineseSimplifiedDepTags.intersection(set(englishDepTags))
  #English:tag_types = ["acl", "acomp", "advcl", "advmod", "agent", "amod", "appos", "attr", "aux", "auxpass", "case", "cc", "ccomp", "compound", "conj", "cop", "csubj", "csubjpass", "dative", "dep", "det", "dobj", "expl", "intj", "mark", "meta", "neg", "nn", "npmod", "nsubj", "nsubjpass", "oprd", "obj", "obl", "pcomp", "pobj", "poss", "preconj", "prep", "prt", "punct",  "quantmod", "relcl", "root", "xcomp"]
  for category in overlapChineseEnglishDepTags:
    depTagDescription = "GrammarRelation:" + category
    linguisticFeatures[depTagDescription] = (df['DEP'] == category).sum()/df.shape[0]

In [ ]:
complexTagsInEnglishGCI = ['csubj', 'csubjpass', 'ccomp', 'xcomp', 'acomp', 'pobj', 'advcl', 'mark', 'acl', 'nounmod', 'complm', 'infmod', 'partmod', 'nmod']

In [ ]:
chineseSimplifiedDepTags.intersection(set(complexTagsInEnglishGCI))

{'acl', 'ccomp', 'mark', 'nmod', 'xcomp'}

In [ ]:
overlapChineseEnglishDepTags.intersection(set(complexTagsInEnglishGCI))

{'acl', 'ccomp', 'mark', 'nmod', 'xcomp'}

In [ ]:
def grammarComplexityIndex(df, linguisticFeatures):
  complexTags = chineseSimplifiedDepTags.intersection(set(complexTagsInEnglishGCI))
  numComplexTags = 0
  for tag in complexTags:
    numComplexTags += (df['DEP'] == tag).sum()
  totalNumTags = len(df['DEP'])
  grammarComplexity = numComplexTags/totalNumTags
  linguisticFeatures['Grammar Complexity Index'] = grammarComplexity

#propositional density

In [ ]:
def propositionalDensity(df, linguisticFeatures):
  pdTags = ['VERB', 'ADJ', 'ADV', 'ADP', 'CONJ', 'CCONJ', 'SCONJ']
  numPDTags = 0
  for tag in pdTags:
    numPDTags += (df['POS'] == tag).sum()
  totalNumPDTags = len(df['POS'])
  propositionalDensity = numPDTags/totalNumPDTags
  linguisticFeatures['Propositional Density'] = propositionalDensity

#Type token ration

In [ ]:
def typeTokenRatio(df, linguisticFeatures):
  numTokens = len(df)
  numTypes = len(set(df['text'].str.lower()))
  linguisticFeatures["# of unique words"] = numTypes
  typeTokenRatio = numTypes/numTokens
  linguisticFeatures["Type Token Ratio"] = typeTokenRatio

In [ ]:
def mov_avg_type_token_ratio(df, window_size=50):
  sum_type_token_ratio = 0
  numWindows = 0
  window_start = 0
  window_end = window_size
  while(window_end <= len(df)):
    currentWindow = df['text'][window_start:window_end]
    numTypes = len(set(df['text'][window_start:window_end].str.lower()))
    window_type_token_ratio = numTypes/window_size
    sum_type_token_ratio += window_type_token_ratio
  #update variables for next iteration
    numWindows +=1
    window_start +=1
    window_end +=1
  if numWindows == 0:
    return pd.NA
  moving_average_type_token_ratio = sum_type_token_ratio/numWindows
  return moving_average_type_token_ratio

In [ ]:
def determine_repetition(currentWord, words):
  for word in words:
      if (word != currentWord):
        return False
  return True

In [ ]:
def repetition_of_word_n(n,df):
  numRepetitions = 0
  first_word = df['text'][0]
  current_word = first_word
  current_word_index = 0
  next_window_start_index = 1
  next_window_end_index = n
  next_words = None
  while(next_window_end_index < len(df)):
    nextWords = list(df['text'][next_window_start_index:next_window_end_index])
    #print(nextWords)
    if (determine_repetition(current_word, nextWords)):
      numRepetitions += 1
    #update for next iteration
    current_word = nextWords[0]
    current_word_index +=1
    next_window_start_index +=1
    next_window_end_index +=1
  return numRepetitions

#Stopwords

In [ ]:
!pip install stopwordsiso


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import stopwordsiso
from stopwordsiso import stopwords
chineseStopWords = stopwords("zh")  # Chinese
print(chineseStopWords)
len(chineseStopWords)

{'譬如', '：', '别处', '年', '那个', '然后', '俺', '省得', '着呢', '随时', '这么', '及其', '该', '六', '尔后', '哪怕', '眨眼', '自家', '另悉', '犹自', '非徒', '含', '纵然', '看', '即或', '怎奈', '能', '各位', '内', '既然', '当地', '故此', '有及', '总之', '矣哉', '此', '还', '哎呀', '谁知', '起见', '他人', '哟', '要不然', '如果', '我', '都', '由', '得', '既是', '各', '用来', '那会儿', '自后', '这样', '为何', '叮咚', '后者', '不只', '哪', '那么', '喽', '中', '孰知', '％', '何况', '啷当', '人们', '７', '￥', '咳', '即使', '以免', '再则', '唯有', '才', '哪些', '何时', '结果', '到', '怎样', '总而言之', '另', '不尽', '基于', '哪儿', '后', '得了', '其二', '就是', '余外', '七', '凭', '反而', '多么', '个别', '一些', '对', '不拘', '就', '自打', '反过来', '吗', '于是', '只是', '俺们', '此地', '即便', '呸', '比方', '替', '替代', '假若', '上下', '再者', '故', '不尽然', '３', '等等', '人', '依据', '甚至', '在于', '这一来', '况且', '焉', '不论', '只要', '果真', '咚', '如此', '比如', '故而', '作为', '一般', '｝', '因而', '旁人', '既往', '则甚', '区', '又及', '可是', '更', '难道说', '两者', '如其', '当', '本人', '却', '之一', '为止', '一切', '此外', '除', '［', '类如', '那么样', '巴', '惟其', '好', '同时', '只怕', '几', '不但', '不怕', '则', '沿', '莫如', '那时', '以为', '虽', '本', '非独', '来自', 

794

In [ ]:
[len(stopword) for stopword in chineseStopWords]

[2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 3,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 4,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 3,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 3,
 1,
 2,
 1,
 2,
 2,
 2,
 3,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 3,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 3,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 4,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 3,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 3,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 3,


In [ ]:
englishStopWords = stopwords("en")
print(englishStopWords)
len(englishStopWords)

{'cg', 'felt', 'hereafter', 'yt', 'giving', 'pm', 'ask', 'wheres', 'u', 'all', 'aint', "why'll", "isn't", 'regards', 'want', 'meanwhile', 'below', 'vol', 'are', 'dm', 'showns', 'approximately', 'ir', 'small', 'means', 'iq', 'gy', 'successfully', 'bd', 'value', 'too', 'thou', 'clear', 'minus', 'younger', 'actually', 'higher', 'nearly', "how'll", 'si', 'gh', 'itself', 'most', 'la', 'pe', 'auth', 'ad', 'my', 'any', 'changes', 'kw', 'call', 'lets', 'beginnings', 'gb', 'present', 'isnt', 'have', 'you', 'thing', 'mustnt', 'youll', 'when', 'mostly', "where'd", 'empty', 'looking', 'respectively', 'inasmuch', 'abst', 'they', 'may', 'showing', 'obviously', 'say', 'myse”', 'nor', "there's", 'took', 'doesnt', 'indicated', 'sure', "that'll", 'pp', 'several', 'st', 'mine', 'pointed', 'wanted', 'announce', 'another', 'possibly', 'pw', 'hu', 'undoing', 'keeps', 'new', 'regardless', 'http', 'seventy', 'specify', "you're", 'mt', 'necessarily', 'lr', 'sl', 'his', 'who', 'added', 'ki', "i'd", 'herein', 'c

1298

Stopwords: words that are commonly used in an language (https://kavita-ganesan.com/what-are-stop-words/)

In [ ]:
def numStopWords(df, stopWords=chineseStopWords):
  result = 0
  for word in list(df['text']):
    if word in stopWords:
      result+=1
  return result/len(df)

In [ ]:
def averageSentenceLength(df, doc, linguisticFeatures):
  numSentences = len(list(doc.sents))
  #numWords = len(df)
  #both implementations achieve same result for demo sample
  averageSentenceLength = len(df)/numSentences#numSentences/len(df)

  # for sent in doc.sents:
  #   numWords += len(sent)
  #avgSentenceLength = numWords/numSentences
  linguisticFeatures['# of words'] = len(df['POS'])#, corrected on 6.25.2022
  #linguisticFeatures['# of words'] = numWords
  linguisticFeatures["Average sentence length"] = averageSentenceLength

#**Word Length by Morphemes**

In [ ]:
!pip install polyglot
#!sudo apt-get install python-numpy libicu-dev
!pip install pyicu #problematic on terminal: use conda install -c conda-forge pyicu
!pip install pycld2
!pip install morfessor
from polyglot.downloader import downloader
from polyglot.text import Text, Word
!polyglot download morph2.en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52558 sha256=8ba2feecc1a4066e75bfaee40404d6ceeb7b36bd4de8b4795a758fcd73237ce5
  Stored in directory: /root/.cache/pip/wheels/aa/92/4a/b172589446ba537db3bdb9a1f2204f27fe71217981c14ac368
Successfully built polyglot
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyicu: filename=PyICU-2.12-cp310-cp310-linux_x86_64.whl size=1754546 sha256=71c1790e5bae36a22d1d78aa53f6224e74771bbe03551e6be6b99cd9e029eca3
  Stored in directory: /root/.cache/pip/wheels/74/60/95/66d97ac2fdc8be8e526c4254047405fe77feaf064282d1ad07
Successfully built pyicu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 48.6 MB/s eta 

In [ ]:
!polyglot download morph2.zh

[polyglot_data] Downloading package morph2.zh to
[polyglot_data]     /root/polyglot_data...


In [ ]:
def wordLengthByMorphemes(df,linguisticFeatures):
  wordsLengthByMophemes = [len(Word(word, language="zh").morphemes) for word in list(df['text'])]
  linguisticFeatures["Word Length by Morphemes Mean"] = np.mean(wordsLengthByMophemes)
  linguisticFeatures["Word Length by Morphemes Standard Deviation"] = np.std(wordsLengthByMophemes)
  #linguisticFeatures["Max Word Length by Morphemes"] = np.max(wordsLengthByMophemes)
  #linguisticFeatures["Min Word Length by Morphemes"] = np.min(wordsLengthByMophemes)

#Word length by phoneme

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model = T5ForConditionalGeneration.from_pretrained('charsiu/g2p_multilingual_byT5_small_100') # 'charsiu/g2p_multilingual_byT5_tiny_16_layers_100'

tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

In [ ]:
# tokenized English words
words = ['Char', 'siu', 'is', 'a', 'Cantonese', 'style', 'of', 'barbecued', 'pork']
words = ['<eng-us>: '+i for i in words]
out = tokenizer(words,padding=True,add_special_tokens=False,return_tensors='pt')
preds = model.generate(**out,num_beams=1,max_length=50) # We do not find beam search helpful. Greedy decoding is enough.
phones = tokenizer.batch_decode(preds.tolist(),skip_special_tokens=True)
print(phones)
# Output: ['ˈtʃɑɹ', 'ˈsiw', 'ˈɪs', 'ˈɑ', 'ˈkæntəˌniz', 'ˈstaɪɫ', 'ˈɑf', 'ˈbɑɹbɪkˌjud', 'ˈpɔɹk']


['ˈtʃɑɹ', 'ˈʃu', 'ˈɪs', 'ˈeɪ', 'ˌkæntəˈniz', 'ˈstaɪɫ', 'ˈəf', 'ˈbɑɹbɪkˌjud', 'ˈpɔɹk']


In [ ]:
def wordLengthByPhonemes(df,linguisticFeatures):
  words = list(df['text'])
  words = ['<zho-s>: '+i for i in words]
  out = tokenizer(words,padding=True,add_special_tokens=False,return_tensors='pt')
  preds = model.generate(**out,num_beams=1,max_length=50) # We do not find beam search helpful. Greedy decoding is enough.
  phones = tokenizer.batch_decode(preds.tolist(),skip_special_tokens=True)
  linguisticFeatures["Word Length by Phonemes Mean"] = np.mean([len(i) for i in phones])
  linguisticFeatures["Word Length by Phonemes Standard Deviation"] = np.std([len(i) for i in phones])
  #linguisticFeatures["Max Word Length by Morphemes"] = np.max(wordsLengthByMophemes)
  #linguisticFeatures["Min Word Length by Morphemes"] = np.min(wordsLengthByMophemes)

#**Word Frequency**

In [ ]:
import pandas as pd

In [ ]:
!ls "gdrive/MyDrive/Colab_Notebooks/AI_aphasia/WordFrequency/"

ls: cannot access 'gdrive/MyDrive/Colab_Notebooks/AI_aphasia/WordFrequency/': No such file or directory


In [ ]:
wordFreqDF = pd.read_csv('SUBTLEX-CH-WF.csv')
wordFreqDF.head()

,Word,WCount,W/million,logW,W-CD,W-CD%,logW-CD
0,的,1682530,50155.13,6.2260,6243,100.00,3.7954
1,我,1682285,50147.83,6.2259,6242,99.98,3.7953
2,你,1329424,39629.27,6.1237,6242,99.98,3.7953
3,是,947807,28253.52,5.9767,6243,100.00,3.7954
4,了,946365,28210.53,5.9761,6243,100.00,3.7954


In [ ]:
chineseWordFreqProbs = dict(zip(wordFreqDF['Word'], wordFreqDF['logW']))

In [ ]:
chineseWordContextualDiversity = dict(zip(wordFreqDF['Word'], wordFreqDF['logW-CD']))

In [ ]:
def wordFreqMetrics(df,linguisticFeatures):
  openClassPOSTags = ['NOUN', 'VERB', 'ADJ', 'ADV']
  openClassDF = df[df.POS.isin(openClassPOSTags)]
  wordFreqVals = []
  wordCDVals = []
  for word in list(openClassDF['text']):
    wordFreqVal = chineseWordFreqProbs.get(word)
    if wordFreqVal != None:
      wordFreqVals.append(wordFreqVal)
    wordCD = chineseWordContextualDiversity.get(word)
    if wordCD != None:
      wordCDVals.append(wordCD)
  linguisticFeatures["Word Frequency Mean"] = np.nanmean(wordFreqVals)
  linguisticFeatures["Word Frequency Standard Deviation"] = np.nanstd(wordFreqVals)
  linguisticFeatures["Word Contextual Diversity Mean"] = np.nanmean(wordCDVals)
  linguisticFeatures["Word Contextual Diversity Standard Deviation"] = np.nanstd(wordCDVals)

#**Age of Acquisition**

In [ ]:
import pandas as pd

Source: https://link.springer.com/article/10.3758/s13428-020-01455-8

In [ ]:
ageOfAcquistionDF = pd.read_csv('Chinese_AOA_13428_2020_1455_MOESM1_ESM.csv')
#del semanticDiversityDF.iloc[0]
ageOfAcquistionDF.head()

,Word ID,Word,AoA Mean,AoA SD,No. of Raters,Male %,Female %,Age Max,Age Min,Education Max,Education Min
0,1,阿胶,14.6,5.5,20,45,55,47,19,5,4
1,17280,阿谀奉承,14.4,6.1,20,35,65,44,18,5,4
2,961,哀悼,14.3,5.2,20,45,55,53,19,4,3
3,19274,哀号,12.3,4.6,27,56,44,54,18,5,3
4,15508,哀求,12.4,4.4,27,59,41,47,20,5,3


In [ ]:
ageOfAcquistionMeanMapping = dict(zip(list(ageOfAcquistionDF['Word']), list(ageOfAcquistionDF['AoA Mean'])))
ageOfAcquistionSDMapping = dict(zip(list(ageOfAcquistionDF['Word']), list(ageOfAcquistionDF['AoA SD'])))

In [ ]:
def compute_AoA_MappingForWordList(df,linguisticFeatures):
  wordsAOAMeanList = []
  wordsAOASDList = []
  openClassPOSTags = ['NOUN', 'VERB', 'ADJ', 'ADV']
  openClassDF = df[df.POS.isin(openClassPOSTags)]
  for word in list(openClassDF['text']):
    wordMeanAOA = ageOfAcquistionMeanMapping.get(word.lower())
    if wordMeanAOA == None:
      wordsAOAMeanList.append(np.nan)
      continue
    wordsAOAMeanList.append(float(wordMeanAOA))
    wordAOASD = ageOfAcquistionSDMapping.get(word.lower())
    if wordAOASD == None:
      wordsAOASDList.append(np.nan)
      continue
    wordsAOASDList.append(float(wordAOASD))
  linguisticFeatures["Age of Acquistion Mean Average"] = np.nanmean(wordsAOAMeanList)
  linguisticFeatures["Age of Acquistion SD Average"] = np.nanmean(wordsAOASDList)

#Yngve and frazier

In [ ]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 27.6 MB/s eta 0:00:00


In [ ]:
import stanza

In [ ]:
stanza.download('zh-hans') # download Chinese simplified
pipe = stanza.Pipeline(lang='zh-hans', processors='tokenize,  pos, lemma, depparse, constituency') # initialize Chinese neural pipeline

INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package          |
-----------------------------------
| tokenize     | gsdsimp          |
| pos          | gsdsimp_charlm   |
| lemma        | gsdsimp_nocharlm |
| constituency | ctb-51_charlm    |
| depparse     | gsdsimp_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
def get_yngve_score(TreeNode, score, leaf_num, total_score):
  if TreeNode.is_leaf():
    total_score += score
  else:
    num_children = len(TreeNode.children)
    for i in range(0, num_children):
      total_score = get_yngve_score(TreeNode.children[i], score + (num_children - 1) - i, i, total_score)
  return total_score

In [ ]:
def get_frazier_score(node, my_score, total_score, sent_parent):
  if node.is_leaf():
    sent_val = 1.5 if sent_parent else 0
    #print("Leaf Node: ", node, " Gets Score: ", my_score + sent_val)
    total_score = total_score + my_score + sent_val
  else:
    num_children = len(node.children)
    for i in range(0, num_children):
      sent_bool = True if (node.label == "S" and i == 0) or sent_parent else False
      sibling_num = 0 if ((i == (num_children - 1)) and (num_children != 1)) else 1
      total_score = get_frazier_score(node.children[i], sibling_num, total_score, sent_bool)
  return total_score



In [ ]:
def yngve_fraz_score_normalized(text,linguisticFeatures):
  numWords = 0
  textYngveScore = 0
  textFrazierScore = 0
  info_map = {}
  for sent in text.sentences:
    stripped_sent = [word.text for word in sent.words if word.upos != "PUNCT"]
    preprocessed = ' '.join(stripped_sent)
    clean_sent = pipe(preprocessed).sentences[0];
    sent_tree_root = clean_sent.constituency.children[0]
    score = 0
    yngve_score = get_yngve_score(sent_tree_root, score, 0, total_score=0)
    #print("Yngve score:", yngve_score)
    fraz_score = get_frazier_score(sent_tree_root, 0, total_score=0, sent_parent=False)
    #print("Frazier score:", fraz_score)
    info_map[sent.text] = [clean_sent.constituency, yngve_score, fraz_score]
    textYngveScore += yngve_score
    textFrazierScore += fraz_score
    numWords += len(stripped_sent)
  linguisticFeatures["Yngve"] = textYngveScore/numWords
  linguisticFeatures["Frazier"] = textFrazierScore/numWords

#Run everything

In [ ]:
def extractLinguisticFeatures(transcription, sampleName):
  import spacy
  nlp =spacy.load('zh_core_web_trf')
  doc = nlp(filterText(transcription))
  df = setupSpacyDF(doc)
  linguisticFeatures = {}
  posNormalizedCounts(df, linguisticFeatures)
  linguisticFeatures['Noun to Verb ratio'] = (df['POS'] == 'NOUN').sum()/(df['POS'] == 'VERB').sum()
  depTagCounts(df, linguisticFeatures)
  grammarComplexityIndex(df, linguisticFeatures)
  propositionalDensity(df, linguisticFeatures)
  typeTokenRatio(df, linguisticFeatures)
  wordLengthByMorphemes(df,linguisticFeatures)
  wordLengthByPhonemes(df,linguisticFeatures)
  linguisticFeatures["Moving Average Type Token Ratio"] = mov_avg_type_token_ratio(df, window_size=50)
  linguisticFeatures["Repetitions of word twice"] = repetition_of_word_n(2,df)
  linguisticFeatures["StopwordsProportion"] = numStopWords(df)
  averageSentenceLength(df, doc, linguisticFeatures)
  wordFreqMetrics(df,linguisticFeatures)
  compute_AoA_MappingForWordList(df,linguisticFeatures)
  yngve_fraz_score_normalized(pipe(transcription),linguisticFeatures)
  return linguisticFeatures

In [ ]:
f = open("chinese_text_demo.txt","r")
transcription = f.read()
feats = extractLinguisticFeatures(transcription, 'Demo')
print(len(feats))
feats

59


{'POS_TAG:ADJ': 0.0,
 'POS_TAG:ADP': 0.02531645569620253,
 'POS_TAG:ADV': 0.13924050632911392,
 'POS_TAG:AUX': 0.0,
 'POS_TAG:CONJ': 0.0,
 'POS_TAG:CCONJ': 0.0,
 'POS_TAG:DET': 0.05063291139240506,
 'POS_TAG:INTJ': 0.012658227848101266,
 'POS_TAG:NOUN': 0.189873417721519,
 'POS_TAG:NUM': 0.08860759493670886,
 'POS_TAG:PART': 0.11392405063291139,
 'POS_TAG:PRON': 0.12658227848101267,
 'POS_TAG:PROPN': 0.0,
 'POS_TAG:SCONJ': 0.012658227848101266,
 'POS_TAG:VERB': 0.24050632911392406,
 'Noun to Verb ratio': 0.7894736842105263,
 'GrammarRelation:case': 0.0379746835443038,
 'GrammarRelation:amod': 0.0,
 'GrammarRelation:neg': 0.0,
 'GrammarRelation:advmod': 0.12658227848101267,
 'GrammarRelation:dobj': 0.06329113924050633,
 'GrammarRelation:acl': 0.0379746835443038,
 'GrammarRelation:cc': 0.012658227848101266,
 'GrammarRelation:det': 0.0379746835443038,
 'GrammarRelation:conj': 0.0379746835443038,
 'GrammarRelation:punct': 0.0,
 'GrammarRelation:nsubjpass': 0.0,
 'GrammarRelation:nummod': 0

In [ ]:
def extractLingFeaturesFromDirectory(inputDirectory, sampleName = ""):
  import os
  fileAddresses = os.listdir(inputDirectory)
  linguisticFeats = []
  for file in fileAddresses:
    sampleID = file.split('_')[-1].split('.')[0]
    f = open(inputDirectory+file,"r")
    transcription = f.read()
    fileLinguisticFeats = extractLinguisticFeatures(transcription, sampleID)
    linguisticFeats.append(fileLinguisticFeats)
  import pandas as pd
  directoryLinguisticFeats = pd.concat(linguisticFeats)
  from google.colab import files
  directoryLinguisticFeats.to_csv(sampleName + "_LinguisticFeatures.csv")
  files.download(sampleName + "_LinguisticFeatures.csv")
  return directoryLinguisticFeats

In [ ]:
!unzip taukadialTranscriptions.zip

In [ ]:
import os
inputDirectory = "trimmedChineseTranscriptions/"
fileAddresses = sorted(os.listdir(inputDirectory))
linguisticFeats = []
for file in fileAddresses:
  sampleID = file.split('_')[-1].split('.')[0]
  print("Working on", sampleID)
  f = open(inputDirectory+file,"r")
  transcription = f.read()
  fileLinguisticFeats = extractLinguisticFeatures(transcription, sampleID)
  linguisticFeats.append(fileLinguisticFeats)

Working on taukdial-003-1
Working on taukdial-003-2
Working on taukdial-003-3
Working on taukdial-004-1
Working on taukdial-004-2
Working on taukdial-004-3
Working on taukdial-006-1
Working on taukdial-006-2
Working on taukdial-006-3
Working on taukdial-007-1
Working on taukdial-007-2
Working on taukdial-007-3
Working on taukdial-009-1
Working on taukdial-009-2
Working on taukdial-009-3
Working on taukdial-021-1
Working on taukdial-021-2
Working on taukdial-021-3
Working on taukdial-023-1
Working on taukdial-023-2
Working on taukdial-023-3
Working on taukdial-024-1
Working on taukdial-024-2
Working on taukdial-024-3
Working on taukdial-025-1
Working on taukdial-025-2
Working on taukdial-025-3
Working on taukdial-027-1
Working on taukdial-027-2
Working on taukdial-027-3
Working on taukdial-039-1
Working on taukdial-039-2
Working on taukdial-039-3
Working on taukdial-044-1
Working on taukdial-044-2
Working on taukdial-044-3
Working on taukdial-045-1
Working on taukdial-045-2
Working on t

<ipython-input-80-fed047322367>:17: RuntimeWarning: Mean of empty slice
  linguisticFeatures["Age of Acquistion Mean Average"] = np.nanmean(wordsAOAMeanList)
<ipython-input-80-fed047322367>:18: RuntimeWarning: Mean of empty slice
  linguisticFeatures["Age of Acquistion SD Average"] = np.nanmean(wordsAOASDList)


Working on taukdial-097-1
Working on taukdial-097-2
Working on taukdial-097-3
Working on taukdial-098-1
Working on taukdial-098-2
Working on taukdial-098-3
Working on taukdial-100-1
Working on taukdial-100-2
Working on taukdial-100-3
Working on taukdial-105-1
Working on taukdial-105-2
Working on taukdial-105-3
Working on taukdial-106-1
Working on taukdial-106-2
Working on taukdial-106-3
Working on taukdial-107-1
Working on taukdial-107-2
Working on taukdial-107-3
Working on taukdial-108-1
Working on taukdial-108-2
Working on taukdial-108-3
Working on taukdial-109-1
Working on taukdial-109-2
Working on taukdial-109-3
Working on taukdial-110-1
Working on taukdial-110-2
Working on taukdial-110-3
Working on taukdial-111-1
Working on taukdial-111-2
Working on taukdial-111-3
Working on taukdial-121-1
Working on taukdial-121-2
Working on taukdial-121-3
Working on taukdial-122-1
Working on taukdial-122-2
Working on taukdial-122-3
Working on taukdial-127-1
Working on taukdial-127-2
Working on t

In [ ]:
fileAddresses = sorted(os.listdir(inputDirectory))
sampleNames = [file.split('_')[-1].split('.')[0] for file in fileAddresses]
sampleNames

In [ ]:
# Convert list of dictionaries to DataFrame
df = pd.DataFrame(linguisticFeats)
df.insert(0, 'SampleID', sampleNames)
# Display the DataFrame
df.head()

,SampleID,POS_TAG:ADJ,POS_TAG:ADP,POS_TAG:ADV,POS_TAG:AUX,POS_TAG:CONJ,POS_TAG:CCONJ,POS_TAG:DET,POS_TAG:INTJ,POS_TAG:NOUN,...,# of words,Average sentence length,Word Frequency Mean,Word Frequency Standard Deviation,Word Contextual Diversity Mean,Word Contextual Diversity Standard Deviation,Age of Acquistion Mean Average,Age of Acquistion SD Average,Yngve,Frazier
0,taukdial-003-1,0.187500,0.062500,0.187500,0.0,0.0,0.0,0.000000,0.0,0.312500,...,16,0.062500,4.002315,1.012446,3.309154,0.446264,8.500,4.400000,1.933333,0.800000
1,taukdial-003-2,0.000000,0.000000,0.115385,0.0,0.0,0.0,0.000000,0.0,0.192308,...,26,0.038462,4.625792,1.234761,3.457069,0.756639,7.100,2.800000,1.117647,0.558824
2,taukdial-003-3,0.000000,0.000000,0.250000,0.0,0.0,0.0,0.041667,0.0,0.166667,...,24,0.041667,4.733700,0.997490,3.612527,0.431003,7.725,3.975000,1.700000,0.600000
3,taukdial-004-1,0.043478,0.043478,0.021739,0.0,0.0,0.0,0.043478,0.0,0.260870,...,46,0.021739,4.499623,1.289841,3.423369,0.878954,9.000,5.400000,2.017544,0.473684
4,taukdial-004-2,0.020000,0.040000,0.100000,0.0,0.0,0.0,0.020000,0.0,0.300000,...,50,0.020000,3.529423,1.381390,2.904505,0.879952,8.500,3.583333,1.694915,0.576271


In [ ]:
from google.colab import files
df.to_csv("taukadialChinese_AllLinguisticFeatures.csv",index=None)
files.download("taukadialChinese_AllLinguisticFeatures.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#blabla feats

In [ ]:
import stanza

 #Download the Chinese model for Stanza
stanza.download('zh-hans')

# Initialize the pipeline
nlp = stanza.Pipeline('zh-hans', processors='tokenize,pos,lemma,depparse', use_gpu=False) #mwt processor doesn't exist

INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...
INFO:stanza:File exists: /root/stanza_resources/zh-hans/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package          |
--------------------------------
| tokenize  | gsdsimp          |
| pos       | gsdsimp_charlm   |
| lemma     | gsdsimp_nocharlm |
| depparse  | gsdsimp_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
f = open("chinese_text_demo.txt","r")
text = f.read()

In [ ]:
doc = nlp(text)

In [ ]:
for sent in doc.sentences:
  #print(sent.text)
  for word in sent.words:
    print(word.text, word.upos, word.misc,word.deprel,word.feats)

'' PUNCT None punct None
'好 ADJ None csubj None
,這邊 ADV None advmod None
有 VERB None root None
一 NUM None nummod NumType=Card
個 NOUN None clf None
小 PART None case None
孩子 NOUN None obj None
玩這 DET None det None
個 NOUN None clf None
叫 VERB None acl:relcl None
什麼 PRON None obj None
? PUNCT None punct None
鮷魚喔鮷魚 NOUN None obj None
的 PART None mark:rel None
那個遊戲那 DET None det None
這 DET None det None
邊 NOUN None nsubj None
就是 AUX None cop None
一 NUM None nummod NumType=Card
個 NOUN None clf None
老闆 NOUN None nsubj None
在 ADV None advmod None
擺麵,擺那個 VERB None parataxis None
飲食 VERB None compound None
攤 PART None obj None
他 PRON None nsubj Person=3
利用 VERB None advcl None
這 DET None det None
個 NOUN None obl None
說 VERB None advcl None
他 PRON None nsubj Person=3
們 PRON None appos Number=Plur|Person=3
跟 ADP None case None
幾 NUM None nummod NumType=Card
個 NOUN None clf None
小孩 NOUN None obl None
在 VERB None advcl None
那邊 PRON None obj None
玩 VERB None ccomp None
Sparat X None obj None
如果 ADP No

In [ ]:
import stanza

 #Download the Chinese model for Stanza
stanza.download('zh-hans')

# Initialize the pipeline
nlp = stanza.Pipeline('zh-hans', processors='tokenize,pos,lemma,depparse', use_gpu=False) #mwt processor doesn't exist
# Process text and extract features
def extract_features(text):
    doc = nlp(text)

    print("Tokens:")
    for sent in doc.sentences:
        print(sent.text)
        for word in sent.words:
            print(word.text, word.xpos, word.misc, word.ufeats)
        print()

    inflected_verbs = 0
    auxiliary_verbs = 0
    gerund_verbs = 0
    participles = 0
    clauses = 0
    noun_phrases = 0
    verb_phrases = 0
    infinitive_phrases = 0
    prepositional_phrases = 0
    dependent_clauses = 0
    discourse_markers = 0

    for sent in doc.sentences:
        # Count verbs and their types
        for word in sent.words:
            if 'VB' in word.xpos:  # VB: Verb, base form
                inflected_verbs += 1
            elif 'VBD' in word.xpos or 'VBN' in word.upos:  # VBD: Verb, past tense, VBN: Verb, past participle
                participles += 1
            elif 'VBG' in word.xpos:  # VBG: Verb, gerund or present participle
                gerund_verbs += 1
            elif 'MD' in word.xpos:  # MD: Modal auxiliary
                auxiliary_verbs += 1

            # Count discourse markers
            if word.deprel == 'discourse':
                discourse_markers += 1

        # Count clauses
        clauses += len(sent.dependencies)

        # Count noun phrases, verb phrases, infinitive phrases, prepositional phrases, and dependent clauses
        for token in sent.tokens:
            if token.misc is not None:
                if 'NP' in token.misc:
                    noun_phrases += 1
                elif 'VP' in token.misc:
                    verb_phrases += 1
                elif 'VBG' in token.misc:  # VB or VBG might indicate infinitive phrase
                    infinitive_phrases += 1
                elif 'PP' in token.misc:
                    prepositional_phrases += 1
                elif 'SBAR' in token.misc:
                    dependent_clauses += 1

    total_tokens = sum(len(sent.words) for sent in doc.sentences)
    clause_rate = clauses / total_tokens
    noun_phrase_rate = noun_phrases / total_tokens
    verb_phrase_rate = verb_phrases / total_tokens
    infinitive_phrase_rate = infinitive_phrases / total_tokens
    prepositional_phrase_rate = prepositional_phrases / total_tokens
    dependent_clause_rate = dependent_clauses / total_tokens
    discourse_marker_rate = discourse_markers / total_tokens

    features = {
        'Proportion of inflected verbs': inflected_verbs / total_tokens,
        'Proportion of auxiliary verbs': auxiliary_verbs / total_tokens,
        'Proportion of gerund verbs': gerund_verbs / total_tokens,
        'Proportion of participles': participles / total_tokens,
        'Number of clauses': clauses,
        'Clause rate': clause_rate,
        'Noun phrase rate': noun_phrase_rate,
        'Verb phrase rate': verb_phrase_rate,
        'Infinitive phrase rate': infinitive_phrase_rate,
        'Prepositional phrase rate': prepositional_phrase_rate,
        'Dependent clause rate': dependent_clause_rate,
        'Discourse marker rate': discourse_marker_rate
    }

    return features
f = open("chinese_text_demo.txt","r")
text = f.read()

# Extract features
features = extract_features(text)

# Print features
for feature, value in features.items():
    print(f'{feature}: {value}')


INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...
INFO:stanza:File exists: /root/stanza_resources/zh-hans/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package          |
--------------------------------
| tokenize  | gsdsimp          |
| pos       | gsdsimp_charlm   |
| lemma     | gsdsimp_nocharlm |
| depparse  | gsdsimp_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


Tokens:
'''
好,這邊有一個小孩子玩
這個叫什麼? 鮷魚喔
鮷魚的那個遊戲
那這邊就是一個老闆在擺麵,擺那個飲食攤


AttributeError: 'Word' object has no attribute 'ufeats'

In [ ]:
import stanza

# Download the Chinese model for Stanza
stanza.download('zh-hans')

# Initialize the pipeline
nlp = stanza.Pipeline('zh-hans', processors='tokenize,pos,lemma,depparse', use_gpu=False) #mwt processor doesn't exist


# Process text and extract features
def extract_features(text):
    doc = nlp(text)

    inflected_verbs = 0
    auxiliary_verbs = 0
    gerund_verbs = 0
    participles = 0
    clauses = 0
    noun_phrases = 0
    verb_phrases = 0
    infinitive_phrases = 0
    prepositional_phrases = 0
    dependent_clauses = 0
    discourse_markers = 0

    for sent in doc.sentences:
        # Count verbs and their types
        for word in sent.words:
            if 'VB' in word.upos:  # VB: Verb, base form
                inflected_verbs += 1
            elif 'VBD' in word.upos or 'VBN' in word.upos:  # VBD: Verb, past tense, VBN: Verb, past participle
                participles += 1
            elif 'VBG' in word.upos:  # VBG: Verb, gerund or present participle
                gerund_verbs += 1
            elif 'MD' in word.upos:  # MD: Modal auxiliary
                auxiliary_verbs += 1

            # Count discourse markers
            if word.deprel == 'discourse':
                discourse_markers += 1

        # Count clauses
        clauses += len(sent.dependencies)

        # Count noun phrases, verb phrases, infinitive phrases, prepositional phrases, and dependent clauses
        for token in sent.tokens:
            if token.misc is not None:
                if 'NP' in token.misc:
                    noun_phrases += 1
                elif 'VP' in token.misc:
                    verb_phrases += 1
                elif 'VBG' in token.misc:  # VB or VBG might indicate infinitive phrase
                    infinitive_phrases += 1
                elif 'PP' in token.misc:
                    prepositional_phrases += 1
                elif 'SBAR' in token.misc:
                    dependent_clauses += 1

    total_tokens = sum(len(sent.words) for sent in doc.sentences)
    clause_rate = clauses / total_tokens
    noun_phrase_rate = noun_phrases / total_tokens
    verb_phrase_rate = verb_phrases / total_tokens
    infinitive_phrase_rate = infinitive_phrases / total_tokens
    prepositional_phrase_rate = prepositional_phrases / total_tokens
    dependent_clause_rate = dependent_clauses / total_tokens
    discourse_marker_rate = discourse_markers / total_tokens

    features = {
        'Proportion of inflected verbs': inflected_verbs / total_tokens,
        'Proportion of auxiliary verbs': auxiliary_verbs / total_tokens,
        'Proportion of gerund verbs': gerund_verbs / total_tokens,
        'Proportion of participles': participles / total_tokens,
        #'Number of clauses': clauses,
        'Clause rate': clause_rate,
        'Noun phrase rate': noun_phrase_rate,
        'Verb phrase rate': verb_phrase_rate,
        'Infinitive phrase rate': infinitive_phrase_rate,
        'Prepositional phrase rate': prepositional_phrase_rate,
        'Dependent clause rate': dependent_clause_rate,
        'Discourse marker rate': discourse_marker_rate
    }

    return features

f = open("chinese_text_demo.txt","r")
text = f.read()

# Extract features
features = extract_features(text)

# Print features
for feature, value in features.items():
    print(f'{feature}: {value}')



INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...
INFO:stanza:File exists: /root/stanza_resources/zh-hans/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package          |
--------------------------------
| tokenize  | gsdsimp          |
| pos       | gsdsimp_charlm   |
| lemma     | gsdsimp_nocharlm |
| depparse  | gsdsimp_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


Proportion of inflected verbs: 0.0
Proportion of auxiliary verbs: 0.0
Proportion of gerund verbs: 0.0
Proportion of participles: 0.0
Number of clauses: 91
Clause rate: 1.0
Noun phrase rate: 0.0
Verb phrase rate: 0.0
Infinitive phrase rate: 0.0
Prepositional phrase rate: 0.0
Dependent clause rate: 0.0
Discourse marker rate: 0.0


In [ ]:
import os
inputDirectory = "trimmedChineseTranscriptions/"
fileAddresses = sorted(os.listdir(inputDirectory))
blablaFeats = []
for file in fileAddresses:
  sampleID = file.split('_')[-1].split('.')[0]
  print("Working on", sampleID)
  f = open(inputDirectory+file,"r")
  transcription = f.read()
  fileBlablaFeats = extract_features(transcription)
  blablaFeats.append(fileBlablaFeats)

Working on taukdial-003-1
Working on taukdial-003-2
Working on taukdial-003-3
Working on taukdial-004-1
Working on taukdial-004-2
Working on taukdial-004-3
Working on taukdial-006-1
Working on taukdial-006-2
Working on taukdial-006-3
Working on taukdial-007-1
Working on taukdial-007-2
Working on taukdial-007-3
Working on taukdial-009-1
Working on taukdial-009-2
Working on taukdial-009-3
Working on taukdial-021-1
Working on taukdial-021-2
Working on taukdial-021-3
Working on taukdial-023-1
Working on taukdial-023-2
Working on taukdial-023-3
Working on taukdial-024-1
Working on taukdial-024-2
Working on taukdial-024-3
Working on taukdial-025-1
Working on taukdial-025-2
Working on taukdial-025-3
Working on taukdial-027-1
Working on taukdial-027-2
Working on taukdial-027-3
Working on taukdial-039-1
Working on taukdial-039-2
Working on taukdial-039-3
Working on taukdial-044-1
Working on taukdial-044-2
Working on taukdial-044-3
Working on taukdial-045-1
Working on taukdial-045-2
Working on t

In [ ]:
fileAddresses = sorted(os.listdir(inputDirectory))
sampleNames = [file.split('_')[-1].split('.')[0] for file in fileAddresses]
sampleNames

In [ ]:
# Convert list of dictionaries to DataFrame
blablaDF = pd.DataFrame(blablaFeats)
blablaDF.insert(0, 'SampleID', sampleNames)
# Display the DataFrame
blablaDF.head()

,SampleID,Proportion of inflected verbs,Proportion of auxiliary verbs,Proportion of gerund verbs,Proportion of participles,Number of clauses,Clause rate,Noun phrase rate,Verb phrase rate,Infinitive phrase rate,Prepositional phrase rate,Dependent clause rate,Discourse marker rate
0,taukdial-003-1,0.0,0.0,0.0,0.0,15,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,taukdial-003-2,0.0,0.0,0.0,0.0,34,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,taukdial-003-3,0.0,0.0,0.0,0.0,30,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,taukdial-004-1,0.0,0.0,0.0,0.0,57,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,taukdial-004-2,0.0,0.0,0.0,0.0,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
blablaDF.describe()

,Proportion of inflected verbs,Proportion of auxiliary verbs,Proportion of gerund verbs,Proportion of participles,Number of clauses,Clause rate,Noun phrase rate,Verb phrase rate,Infinitive phrase rate,Prepositional phrase rate,Dependent clause rate,Discourse marker rate
count,198.0,198.0,198.0,198.0,198.000000,198.0,198.0,198.0,198.0,198.0,198.0,198.000000
mean,0.0,0.0,0.0,0.0,106.252525,1.0,0.0,0.0,0.0,0.0,0.0,0.006678
std,0.0,0.0,0.0,0.0,58.827174,0.0,0.0,0.0,0.0,0.0,0.0,0.010756
min,0.0,0.0,0.0,0.0,15.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,0.0,0.0,0.0,0.0,65.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
50%,0.0,0.0,0.0,0.0,93.500000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
75%,0.0,0.0,0.0,0.0,138.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.011309
max,0.0,0.0,0.0,0.0,343.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.067797


In [ ]:
blablaDF =